## Video

Makes a little video of what happens when $a$ increases in the Pareto distribution of radii
$$
p_Y(x) = \frac{a m^a}{x^{a+1}}
$$
for $x \geq 1$, $p_Y(x) = 0$ for $x < 1$.

If $a > 2$ this has second moment
$$
\mathbb{E}[Y^2] = \frac{a m^a}{a-2}.
$$

For $a > 2$ there are two things we may be interested in doing:
1. Keeping $m=1$, so all the radii are supported on $(1,\infty)$, or
2. Choosing $m = m(a)$ such that $\mathbb{E}[Y^2] = 1$ for all $a > 2$.
In this case, the radii are supported on $(m,\infty)$.

Below I've gone with the second method. It does lead to something funny happening around $a=2$ though, since $m(a) \to 0$ when $a \downarrow 2$.

In [13]:
import numpy as np
from scipy.spatial import KDTree
from tqdm import tqdm, trange
import networkx
import colorspace
from PIL import Image, ImageColor, ImageDraw
import sys, os
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import clear_output

from multiprocessing import Pool

from unconstrained import sample_points
from draw_jm import get_adjacency, colour_graph, get_ball_pixels#, assign_cells_random_radii

In [5]:
def assign_cells_random_radii(seeds, rates, img_size, T=1.0):
    min_cov_times = np.full((img_size,img_size),np.inf) # running minimum coverage times
    assignments = np.full((img_size,img_size),-1,dtype=int)
    for i in range(len(rates)):
        xi = seeds[i]
        gi = rates[i]
        gi2 = gi*gi
        indices, d2s = get_ball_pixels(xi, T*gi, img_size)
        for k, ij_pair in enumerate(indices):
            cov_time2 = d2s[k] / gi2
            if cov_time2 < min_cov_times[ij_pair]:
                assignments[ij_pair] = i
                min_cov_times[ij_pair] = cov_time2
    if -1 in assignments: # i.e. if there were uncovered points.
        # print("Uncovered points found - trying again with a bigger radius.")
        return assign_cells_random_radii(seeds,rates,img_size,2*T) # This is slow compared to Moulinec's method of assigning the remaining unassigned pixels individually.
    return assignments, min_cov_times

In [6]:
def pareto_frame(seeds, a, m, fileprefix):
    rates = m*(np.random.pareto(a,size=seeds.shape[0])+1)
    if a <= 2:
        second_moment = 1
    else:
        second_moment = a*m**a / (a - 2)
    max_time = 2*np.sqrt( np.log(n) / (np.pi * n * second_moment) )
    I = assign_cells_random_radii(seeds, rates, resolution, T = max_time)

    cell_structure = get_adjacency(I)
    print(cell_structure)

    colours = colour_graph(cell_structure)
    print(f'We have a {max(colours.values())+1}-colouring of the cells.')

    c = colorspace.hcl_palettes().get_palette(name="SunsetDark")
    hex_colours = c(max(colours.values())+1)
    rgb_colours = [ImageColor.getcolor(col,"RGB") for col in hex_colours]

    data = np.empty((resolution, resolution, 3), dtype=np.uint8)
    N = I.shape[0]
    for i in range(N):
        for j in range(N):
            data[i,j,:] = rgb_colours[colours[I[i,j]]]

    image = Image.fromarray(data)
    image.save(fileprefix+str(a)+'.png')

If we're not careful, the colours will "flicker" in the video because the adjacency graph changes when the radii change. So we create a supremum of the adjacency graphs, in which two vertices are connected if the corresponding cells are adjacent in _any_ diagram. Then we can colour according to this for each diagram.

That graph isn't planar any more, so the number of colours may grow fast.

A disadvantage of this method is there's a very long wait before any frames appear. Or maybe that's an advantage: sometimes I spend far too long checking "how are my frames coming along?"

---

One interesting consequence of the phase transition around $a=2$: it's pretty noticeable that the assignment runs much faster when $a > 2$ than for $a \leq 2$.

In [8]:
from pylatex import Document, Command, Math, NoEscape, Package
from pdf2image import convert_from_path

u = np.random.random()

geometry_options = {"rmargin":"11cm"}
doc = Document('basic',geometry_options=geometry_options)
doc.append(NoEscape("Each ball has growth rate $$Y_i = m U_i^{-1/a}$$ where the $U_i$s are iid $U(0,1]$ random variables."))
doc.generate_pdf("latex/testdocument",clean_tex=True)
dpi_factor = 1.35
images = convert_from_path("latex/testdocument.pdf",dpi=200*dpi_factor)
im = images[0].crop([int(250*dpi_factor),int(270*dpi_factor),int(820*dpi_factor),int(430*dpi_factor)])
im.save('latex/yi-def.PNG')

In [9]:
def create_latex(a,outname,dpi_factor=1.35):
    geometry_options = {"rmargin":"11cm"}
    doc = Document('basic',geometry_options=geometry_options)
    doc.packages.append(Package('amssymb'))
    if a <= 2:
        doc.append(NoEscape(f'Currently $a = {a:.3f}$,' + ' so $$\mathbb{E}[ Y_i^{' + f'{a:.3f}' + ' - \\varepsilon} ] < \\infty,$$ but $$\mathbb{E}[ Y_i^{' + f'{a:.3f}' + '}] = \\infty.$$'))
        doc.append(NoEscape("Since we have are no second moments, we let $m = 1$."))
    else:
        doc.append(NoEscape(f'Currently $a = {a:.3f}$,' + ' so $$\mathbb{E}[ Y_i^{' + f'{2}' + ' + \\varepsilon} ] < \\infty,$$ but $$\mathbb{E}[ Y_i^{' + f'{a:.3f}' + '}] = \\infty.$$'))
        doc.append(NoEscape("The radii have second moments, so we choose $m = m(a)$ so that $\mathbb{E}[Y_i^2] = 1$."))
    doc.generate_pdf("latex/temppdf",clean_tex=True)
    conv = convert_from_path("latex/temppdf.pdf",dpi=200*dpi_factor)
    x1, y1, x2, y2 = 340, 350, 1140, 800
    im = conv[0].crop([x1,y1,x2,y2])
    im.save(f'latex/{outname}.png')
    return

create_latex(2.4,"testpng")

In [10]:
def add_frame_metadata(im, a, m=1.0, panel_width = 840, font_size=20,dpi_factor=1.35):
    """
    Given an NxN image representing the tessellation where each random radius
    is Pareto distributed with scale m and shape a,
    extends the right-hand side of the image by adding a panel containing
    the values of a and m,
    and saying something about the moments.
    """
    height = im.height
    width = im.width
    expanded_im = Image.new(im.mode, (width+panel_width,height), (255,255,255))
    expanded_im.paste(im,(0,0))
    # Now let's write the metadata
    draw = ImageDraw.Draw(expanded_im)
    draw.text((width+0.05*panel_width,0.025*height),"Model parameters",fill="black",font_size=2.5*font_size)
    # draw.text((width+0.05*panel_width,0.15*height),"Each ball has growth rate",font_size=font_size,fill="black")
    yi_def = Image.open('latex/yi-def.PNG')
    expanded_im.paste(yi_def,(int(width+0.03*panel_width),int(0.12*height)))
    create_latex(a,'temp',dpi_factor)
    moments = Image.open('latex/temp.png')
    expanded_im.paste(moments,(int(width+0.03*panel_width),int(0.35*height)))
    # draw.text((width+0.5*panel_width,0.2*height),f'a = {a:.4f}', fill="black",font_size=font_size)
    return expanded_im

In [11]:
from multiprocessing import Pool

In [ ]:
# Delete the old frames:
os.system('rm -f frames/pareto-*')

fileprefix = "frames/pareto-"
n = 100
resolution = 1080
nframes = 281
exponents = np.linspace(3,0.2,num=nframes,endpoint=True)
RANDOMSEED = 20240422

np.random.seed(RANDOMSEED)

seeds = sample_points(n)

max_time = 2*np.sqrt( np.log(n) / (np.pi * n) )

U = np.random.random(size=seeds.shape[0])


def getassignments(a):
    n = 100
    resolution = 1080
    np.random.seed(RANDOMSEED)
    seeds = sample_points(n)
    U = np.random.random(size=seeds.shape[0])
    max_time = 2*np.sqrt( np.log(n) / (np.pi * n) )
    if a>2:
        m = ((a-2.0)/a)**(1/a)
    else:
        m = 1.0
    rates = m*U**(-1/a)
    return assign_cells_random_radii(seeds, rates, resolution, T=max_time)

with Pool(5) as p:
    assignments,_ = p.map(getassignments,exponents)

# assignments = []
# progress = tqdm(exponents, leave=False)
# for a in progress:
#     progress.set_description(f'Working on a={a:.4f}')
#     if a>2:
#         m = ((a-2.0)/a)**(1/a)
#     else:
#         m = 1.0
#     np.random.seed(RANDOMSEED)
#     rates = m*U**(-a)
#     assignments.append(assign_cells_random_radii(seeds, rates, resolution, T = max_time))

print("Assignments done (that's the hard part over)! Now finding the 'all-time adjacency graph'...")
supG = get_adjacency(assignments[0])
for i in tqdm(range(1,len(assignments)),leave=False):
    supG.update(get_adjacency(assignments[i]))
print(supG)
colours = colour_graph(supG)
print(f'We have a {max(colours.values())+1}-colouring of the cells.')
# It's all deterministic after this point.
c = colorspace.hcl_palettes().get_palette(name="SunsetDark")
hex_colours = c(max(colours.values())+1)
rgb_colours = [ImageColor.getcolor(col,"RGB") for col in hex_colours]
print("Drawing the frames.")
for i in trange(len(exponents),leave=False):
    a = exponents[i]
    I = assignments[i]
    data = np.empty((resolution, resolution, 3), dtype=np.uint8)
    for x in range(resolution):
        for y in range(resolution):
            data[x,y,:] = rgb_colours[colours[I[x,y]]]
    image = Image.fromarray(data)
    image = add_frame_metadata(image,a)
    # image.show()
    image.save(fileprefix+f'{a:.5f}.png')
print("Done! Go and find your frames in the frames/ folder")